### This script provides a framework for loading the data, creating new features from it and generating the predictions file. You can use it to play around with different feature combinations and classifiers to see what works best

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
from sklearn import preprocessing, metrics, ensemble

In [3]:
%matplotlib inline

### Read the data

In [4]:
weather = pd.read_csv('input_data/weather_clean.csv')
train = pd.read_csv('input_data/train.csv')
test = pd.read_csv('input_data/test.csv')
sample = pd.read_csv('input_data/sampleSubmission.csv')

### Drop unnecessary columns

In [5]:
weather = weather.drop(['Depart', 'DewPoint', 'WetBulb', 'CodeSum', 'Heat', 'Cool',
                        'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed', 'Sunrise', 'Sunset'], axis=1)
train = train.drop(['NumMosquitos', 'Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy', 'Trap'], axis=1)
test = test.drop(['Id', 'Address', 'Block', 'Street', 'AddressNumberAndStreet', 'AddressAccuracy', 'Trap'], axis=1)

### Convert Date strings to DateTime

In [6]:
weather['Date'] = weather['Date'].apply(pd.to_datetime)
# spray['Date'] = spray['Date'].apply(pd.to_datetime)
train['Date'] = train['Date'].apply(pd.to_datetime)
test['Date'] = test['Date'].apply(pd.to_datetime)

# Weather-related features

## New Feature: accumulated degree weeks with base temperature = 72 F

In [7]:
# Split data for each station
weather_st1 = weather[weather['Station'] == 1]
weather_st2 = weather[weather['Station'] == 2]

# Restore indexing
weather_st1 = weather_st1.reset_index(drop=True)
weather_st2 = weather_st2.reset_index(drop=True)

# Remove the station row
weather_st1 = weather_st1.drop('Station', axis=1)
weather_st2 = weather_st2.drop('Station', axis=1)

In [8]:
weather_st1['Heat'] = 0
weather_st2['Heat'] = 0
weather_st1['CumulativeHeat'] = 0
weather_st2['CumulativeHeat'] = 0

t_base = 72 #degrees Fahrenheit
weather_st1['Heat'] = weather_st1['Tavg'] - t_base
weather_st2['Heat'] = weather_st2['Tavg'] - t_base

In [9]:
# Accumulate degree days for st1 & st2
for index, row in weather_st1.iterrows():
    year = row['Date'].year
    first_row_of_year = weather_st1['Date'][weather_st1['Date'] == pd.datetime(year, 5, 1)].index[0]
    weather_st1.loc[index, 'CumulativeHeat'] = weather_st1['Heat'][first_row_of_year:index].sum()

for index, row in weather_st2.iterrows():
    year = row['Date'].year
    first_row_of_year = weather_st2['Date'][weather_st2['Date'] == pd.datetime(year, 5, 1)].index[0]
    weather_st2.loc[index, 'CumulativeHeat'] = weather_st2['Heat'][first_row_of_year:index].sum()


# Transform into degree weeks
weather_st1['CumulativeHeat'] /= 7
weather_st2['CumulativeHeat'] /= 7

##New Feature: total degree days over last week

In [10]:
weather_st1['HeatWeek'] = 0
weather_st2['HeatWeek'] = 0

N_days = 7

for index, row in weather_st1.iterrows():
    weather_st1.loc[index, 'HeatWeek'] = weather_st1['Heat'][max(index-N_days, 0):index].sum()

for index, row in weather_st2.iterrows():
    weather_st2.loc[index, 'HeatWeek'] = weather_st2['Heat'][max(index-N_days, 0):index].sum()

In [11]:
# 'Heat' is not needed anymore
weather_st1 = weather_st1.drop('Heat', axis=1)
weather_st2 = weather_st2.drop('Heat', axis=1)

## New Feature: total precipitation over last 7 days

In [12]:
weather_st1['CumulativePrecip'] = 0
weather_st2['CumulativePrecip'] = 0

N_days = 14

# The predictions only begin in June while the weather is from May, 
# so it doesn't matter that calculation is overlapping on the year boundaries

for index, row in weather_st1.iterrows():
    weather_st1.loc[index, 'CumulativePrecip'] = weather_st1['PrecipTotal'][max(index-N_days, 0):index].sum()

for index, row in weather_st2.iterrows():
    weather_st2.loc[index, 'CumulativePrecip'] = weather_st2['PrecipTotal'][max(index-N_days, 0):index].sum()


## New Feature: average temperature over last 14 days

In [13]:
weather_st1['TavgOverNDays'] = 0
weather_st2['TavgOverNDays'] = 0

N_days = 14

# The predictions only begin in June while the weather is from May, 
# so it doesn't matter that calculation is overlapping on the year boundaries


for index, row in weather_st1.iterrows():
    weather_st1.loc[index, 'TavgOverNDays'] = weather_st1['Tavg'][max(index-N_days, 0):index].sum()/N_days

for index, row in weather_st2.iterrows():
    weather_st2.loc[index, 'TavgOverNDays'] = weather_st2['Tavg'][max(index-N_days, 0):index].sum()/N_days

### Merge the data from 2 stations

In [14]:
weather = weather_st1.merge(weather_st2, on='Date')

# Training set related features

## New Feature: amount of days that passed since 01/06 of the respective year

In [15]:
weather['DayNumber'] = 0
for index, row in weather.iterrows():
    year = row['Date'].year
    first_day_of_summer = pd.datetime(year, 6, 1)
    weather.loc[index, 'DayNumber'] = (row['Date'] - first_day_of_summer).days
    

### Append merged data to the training and testing sets

In [16]:
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')

# Average the weather parameters by distance to the station

In [18]:
def interpolate_params(data, attributes):
    # This version uses vectorization and is a lot faster
    # Chicago is small enough that we can treat coordinates as rectangular.
    station = np.array([[41.995, -87.933],
                         [41.786, -87.752]])
    # The distances from each training example to the weather stations:
    data['Dist1'] = (((1 - data[['Latitude', 'Longitude']] + station[0]))**2).sum(axis=1)**0.5
    data['Dist2'] = (((1 - data[['Latitude', 'Longitude']] + station[1]))**2).sum(axis=1)**0.5
    data['TotDist'] = data['Dist1'] + data['Dist2']
         
    # Take the weighted average of the attributes
    for attr in attributes:
        data[attr] = data[attr + '_x']*data['Dist1']/data['TotDist'] + data[attr + '_y']*data['Dist2']/data['TotDist']
        # Data from 2 stations is no longer needed
        data = data.drop([attr + '_x', attr + '_y'], axis=1)
    
    data = data.drop(['Dist1', 'Dist2', 'TotDist'], axis=1)

    return data

In [19]:
columns = ['Tmax', 'Tmin', 'PrecipTotal', 'Tavg', 'CumulativeHeat', 'CumulativePrecip', 'TavgOverNDays', 'HeatWeek']
train = interpolate_params(train, columns)
test = interpolate_params(test, columns)

## New Feature: number of rows for the same species on the same day

grouped = train.groupby(by=['Date'])
count = (grouped.size() - grouped.size().min()) / (grouped.size().max() - grouped.size().min())
count = pd.DataFrame(count).reset_index()
count.columns = ['Date', 'NormalizedCount']
train = train.merge(pd.DataFrame(count), on='Date')

In [22]:
grouped = test.groupby(by=['Date'])
count = (grouped.size() - grouped.size().min()) / (grouped.size().max() - grouped.size().min())
count = pd.DataFrame(count).reset_index()
count.columns = ['Date', 'NormalizedCount']
count
test = test.merge(pd.DataFrame(count), on='Date')

## New Feature: replace the species name by 1-of-N species vectors

In [24]:
num_species = 6
def species_vector(species):
    species_map = {'CULEX RESTUANS' : "100000",
                  'CULEX TERRITANS' : "010000",
                  'CULEX PIPIENS'   : "001000",
                  'CULEX PIPIENS/RESTUANS' : "101000",
                  'CULEX ERRATICUS' : "000100",
                  'CULEX SALINARIUS': "000010",
                  'CULEX TARSALIS' :  "000001",
                  'UNSPECIFIED CULEX': "001100"}
    return pd.Series([b for b in species_map[species]])

In [25]:
for i in range(num_species):
    train['s' + str(i)] = 0
    test['s' + str(i)] = 0

In [26]:
train[['s0', 's1', 's2', 's3', 's4', 's5']] = train['Species'].apply(species_vector)
test[['s0', 's1', 's2', 's3', 's4', 's5']] = test['Species'].apply(species_vector)

In [27]:
train = train.drop('Species', axis=1)
test = test.drop('Species', axis=1)

# Get rid of unnecessary features

In [28]:
train = train.drop(['Tmax', 'Tmin', 'Tavg', 'CumulativeHeat', 'PrecipTotal', 's5'], axis=1) 
test = test.drop(['Tmax', 'Tmin', 'Tavg', 'CumulativeHeat', 'PrecipTotal', 's5'], axis=1) 

In [29]:
train.head(1)

,Date,Latitude,Longitude,WnvPresent,DayNumber,CumulativePrecip,TavgOverNDays,HeatWeek,NormalizedCount,s0,s1,s2,s3,s4
0,2007-05-29,41.95469,-87.800991,0,-3,1.541767,64.211675,-26.030463,0.03663,1,0,1,0,0


In [31]:
test.head(1)

,Date,Latitude,Longitude,DayNumber,CumulativePrecip,TavgOverNDays,HeatWeek,NormalizedCount,s0,s1,s2,s3,s4
0,2008-06-11,41.95469,-87.800991,10,3.793777,68.674002,11.969537,0,1,0,1,0,0


####Checking model quality


In [32]:
clf = ensemble.RandomForestClassifier(n_jobs=-1, n_estimators=1000, min_samples_split=1) 

In [33]:
def test_classifier(clf, train):
    """Leaves one year out and trains the classifier on the rest
       then makes the prediction for the remaining year"""
    years = [2007, 2009, 2011, 2013]
    accuracies = []

    for year in years:
        # Split the training set
        new_train = train[train['Date'].apply(lambda x: x.year) != year]
        train_target = new_train['WnvPresent']
        new_train = new_train.drop('WnvPresent', axis=1)
        
        test = train[train['Date'].apply(lambda x: x.year) == year]
        test_target = test['WnvPresent']
        test = test.drop('WnvPresent', axis=1)
    
        # Train the classifier
        clf.fit(new_train.drop('Date', axis=1), train_target)
        
        # Make predictions for the left-out year
        predictions = clf.predict_proba(test.drop('Date', axis=1))[:,1]
        
        # Calculate AUC
        accuracies.append(metrics.roc_auc_score(test_target, predictions))
        
    return {'for_separate_years' : accuracies, 'average': float(sum(accuracies))/len(accuracies)}

In [35]:
test_classifier(clf, train)

{'average': 0.6883504478992368,
 'for_separate_years': [0.68141045395282684,
  0.68286523483596884,
  0.63436821899515938,
  0.75475788381299225]}

In [51]:
clf.feature_importances_

array([  1.05050752e-01,   1.19593085e-01,   5.14281597e-02,
         8.83329556e-02,   1.76840189e-01,   1.32190100e-01,
         1.61600066e-01,   1.12707725e-01,   2.87670191e-03,
         4.82051050e-02,   0.00000000e+00,   1.17331392e-03,
         1.84622331e-06])

### Train the classifier on all the data

In [26]:
train = train.drop('Date', axis=1)
test = test.drop('Date', axis=1)
clf.fit(train.drop('WnvPresent', axis=1), train['WnvPresent'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=1,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

### Generate the predictions and write them to the output file

In [27]:
file_name = 'temp_over_14_onehot.csv'
predictions = clf.predict_proba(test)[:,1]
sample['WnvPresent'] = predictions

In [28]:
sample.to_csv(file_name, index=False)